In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from bs4 import BeautifulSoup
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from model_utils import create_and_restore_model
from utils import load_image_into_numpy_array#, plot_detections

%matplotlib inline

num_classes = 1
pipeline_config = '/home/evan/Desktop/Tensorflow/models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = './fine_tuned_ckpts/ckpt-1' #set to latest ckpt
(orig_height, orig_width) = (544,960)
(new_height, new_width) = (640,640)
hand_class_id = 1
num_classes = 1
category_index = {hand_class_id: {'id': hand_class_id, 'name': 'hand'}}

In [ ]:
detection_model = \
create_and_restore_model(pipeline_config = pipeline_config, 
                        checkpoint_path = checkpoint_path, 
                        new_height = new_height, 
                        new_width = new_width, 
                        hand_class_id = hand_class_id, 
                        num_classes = num_classes,
                        restore_classification_head = True)

In [ ]:
test_image_dir = '/home/evan/Datasets/Hand Images/test'
test_images_np = []
num_test_frames = len(os.listdir(test_image_dir))
for i in range(1, num_test_frames):
  image_path = os.path.join(test_image_dir, 'frame' + str(i) + '.jpg')
  test_images_np.append(np.expand_dims(
      load_image_into_numpy_array(image_path,new_height,new_width,swap=False), axis=0))

@tf.function
def detect(input_tensor):
  """Run detection on an input image.

  Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

  Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
  """
  preprocessed_image, shapes = detection_model.preprocess(input_tensor)
  prediction_dict = detection_model.predict(preprocessed_image, shapes)
  return detection_model.postprocess(prediction_dict, shapes)

In [ ]:
def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_save_name=None,
                    min_score_thresh=0.9):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_save_name: a name for the image file.
  """
  print(boxes)
  image_np_with_annotations = image_np.copy()
  image_np_with_annotations = \
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image = image_np_with_annotations,
      boxes = boxes,
      classes = classes,
      scores = scores,
      category_index = category_index,
      min_score_thresh = min_score_thresh,
      use_normalized_coordinates=True)
  if image_save_name:
    plt.imsave(image_save_name, image_np_with_annotations)
  else:
    return image_np_with_annotations

In [ ]:
import cv2
kalman = cv2.KalmanFilter(12,4)#track x,y,h,w plus respective velocities and accels in that order
kalman.measurementMatrix= np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                                    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                                    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                                    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
                                    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.identity(12, np.float32) * 1e-5
kalman.measurementNoiseCov = np.identity(4, np.float32) * 1e-2

last_measurement = None
last_box = None
last_score = None
last_class = None
last_prediction = None
first_good_detection = False
label_id_offset = 1
kalman_counter = 1
label_id_offset = 1

for i in range(len(test_images_np)):
  input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
  detections = detect(input_tensor)
  box = detections['detection_boxes'][0][0].numpy()#[ymin, xmin, ymax, xmax]
  class_ = detections['detection_classes'][0][0].numpy()
  if class_ is None:
    class_ = np.array([0])
  score = detections['detection_scores'][0][0].numpy()
  #bbox center coords, height, width
  y = (box[2]-box[0])/2 + box[0]
  x = (box[3]-box[1])/2 + box[1]
  h = box[2]-box[0]
  w = box[3]-box[1]

  if last_measurement is None:
    #initialize states
    kalman.statePre = np.array([[y], [x], [h], [w], [0], [0], [0], [0], [0], [0], [0], [0]], np.float32)
    kalman.statePost = np.array([[y], [x], [h], [w], [0], [0], [0], [0], [0], [0], [0], [0]], np.float32)
  else:
    kp = kalman.predict().reshape(12)[:4]
    kalman.correct(np.array([[y], [x], [h], [w]], np.float32)) #TODO how to weight by confidence in last detection?
    if score < 0.9: #only substitute for kalman when needed 
      print('substituting a low-quality detection for Kalman prediction')
      y,x,h,w = kp
      box = np.array([y-h/2, x-w/2, y+h/2, x+w/2])
      class_ = last_class

  #TODO: look @ below line
  last_class = class_
  last_measurement = np.array([[y], [x], [h], [w]], np.float32)
  # box = np.array([1,1,1,1]) #TEST
  # print(box[np.newaxis, :])
  # print(score)
  plot_detections(
      test_images_np[i][0],
      box[np.newaxis, :],
      np.array([class_.astype(np.uint32) + label_id_offset]),
      np.array([score]),
      category_index, 
      figsize=(15, 20), 
      image_save_name="/home/evan/Desktop/few shot object detection/test_detection_images/gif_frame_" + ('%02d' % i) + ".jpg",
      min_score_thresh=0.0)

In [ ]:
imageio.plugins.freeimage.download()

anim_file = '/home/evan/Desktop/few shot object detection/test_detection_videos/hand_test.gif'

filenames = glob.glob('/home/evan/Desktop/few shot object detection/test_detection_images/gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []
for filename in filenames:
  image = imageio.imread(filename)
  images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=20)

display(IPyImage(open(anim_file, 'rb').read()))